# IEEE Access Paper: A Deep Learning Approach Based on Explainable Artificial Intelligence for Skin Lesion Classification
**By: University of Engineering and Technology,Lahore,pakistan**

# Step-1: Image Preprocessing    
To enhance image quality and focus on the lesion areas, the following preprocessing steps are applied:​

* **Region of Interest (ROI) Extraction:** Center cropping to isolate the lesion.​

* **Resizing:** Adjusting images to 224×224 pixels.​
challenge.isic-archive.com
+1
Kaggle
+1

* **Zero Padding:** Maintaining aspect ratio without distortion.​

* **Noise Reduction:** Applying Gaussian filters to remove artifacts.​

* **Normalization:** Scaling pixel values to the [0,1] range.

In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt

def preprocess_image(image_path):
    # Load image
    img = Image.open(image_path).convert('RGB')
    img_np = np.array(img)

    # Step 1: Center Crop (ROI Extraction)
    h, w = img_np.shape[:2]
    side = min(h, w)
    startx = w//2 - side//2
    starty = h//2 - side//2
    cropped = img_np[starty:starty+side, startx:startx+side]

    # Step 2: Resize to 224x224
    resized = cv2.resize(cropped, (224, 224), interpolation=cv2.INTER_AREA)

    # Step 3: Zero Padding (if needed)
    top, bottom, left, right = (0, 0, 0, 0)
    max_side = max(resized.shape[:2])
    delta_w = max_side - resized.shape[1]
    delta_h = max_side - resized.shape[0]
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    padded = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

    # Step 4: Gaussian Noise Reduction
    denoised = gaussian_filter(padded, sigma=1)

    # Step 5: Normalize to [0,1]
    normalized = denoised / 255.0

    return normalized


# Step-2: Data Augmentation
To increase data diversity and prevent overfitting, the following augmentation techniques are applied:​

* **Rotation:** Random rotations at various angles.​

* **Flipping:** Horizontal and vertical flips.​

* **Cropping:** Random crops to simulate zoom.​

* **Brightness and Contrast Adjustment:** Randomly altering brightness and contrast levels.​

* **Noise Addition:** Introducing random noise to images.

In [2]:
from torchvision import transforms

data_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
])


# Step-3: Model Training with ResNet-18
Utilizing transfer learning, a pre-trained ResNet-18 model is fine-tuned for skin lesion classification. The final fully connected layer is modified to output predictions for the nine classes present in the ISIC 2019 dataset.

In [3]:
import torch
import torch.nn as nn
from torchvision import models

# Load pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

# Freeze early layers
for param in model.parameters():
    param.requires_grad = False

# Modify the final layer for 9-class classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 9)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 156MB/s] 


# Model Explainability with the LIME
To interpret the model's predictions, LIME (Local Interpretable Model-Agnostic Explanations) is employed. LIME provides visual explanations by highlighting regions in the image that most influenced the model's decision.

In [5]:
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt

def explain_prediction(model, image_tensor):
    model.eval()
    image = image_tensor.numpy().transpose(1, 2, 0)

    explainer = lime_image.LimeImageExplainer()
    explanation = explainer.explain_instance(
        image, 
        classifier_fn=lambda x: model(torch.tensor(x).permute(0, 3, 1, 2).float()).detach().numpy(),
        top_labels=1,
        hide_color=0,
        num_samples=1000
    )

    temp, mask = explanation.get_image_and_mask(
        explanation.top_labels[0],
        positive_only=True,
        num_features=5,
        hide_rest=False
    )

    plt.imshow(mark_boundaries(temp / 255.0, mask))
    plt.title('LIME Explanation')
    plt.axis('off')
    plt.show()
